In [1]:
import ant_colony


In [2]:
from ant_colony.aco_tsp import *

In [3]:
from ant_colony.utils import *
from ant_colony.aco_tsp_oo import *

In [4]:
# !pip install line-profiler
# !pip install memory_profiler

In [7]:
import timeit
import cProfile
import pstats
import memory_profiler
from memory_profiler import memory_usage
import line_profiler

### En primer lugar, veamos las características de la máquina que se utilizará para el perfilamiento: 

In [11]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3099.200
BogoMIPS:            4999.96
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

In [17]:
#Este comando devuelve error
#%%bash
#sudo lshw -C memory

### Cargamos unos datos de prueba para el perfilamiento

In [21]:
path = 'gr17_d_city_distances.txt'

In [22]:
G = read_data(path)


In [23]:
G

In [25]:
colonia = colony(G, 
                 init_node=7, 
                 n_ants=5, 
                 max_iter=500, 
                 rho=.5, 
                 alpha=3,
                 beta=3)

# A. Perfilamiento de tiempos de ejecucion

### Utilizamos Timeit para medir tiempos de ejecución

## Probamos el metodo solve.tsp ya que es el método principal de nuestro algoritmo que a su vez llama a los otros métodos de mayor importancia :

### Primero medimos el tiempo que toma ejecutar el método 

In [26]:
start = timeit.timeit()
colonia.solve_tsp()
end = timeit.timeit()
print(f"- Tiempo total: {(end-start)} segundos.")

- Tiempo total: -0.0003791760000240174 segundos.


### A continuación utilizamos cProfile para ver las partes del método que más se tardan 

In [27]:
cprof = cProfile.Profile()
cprof.enable()
res =  colonia.solve_tsp()
cprof.disable()
cprof.print_stats(sort='cumtime')

         3141046 function calls (3140546 primitive calls) in 4.248 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    4.247    2.124 interactiveshell.py:3288(run_code)
        2    0.000    0.000    4.247    2.124 {built-in method builtins.exec}
        1    0.000    0.000    4.247    4.247 <ipython-input-27-5fb9737eed66>:3(<module>)
        1    0.010    0.010    4.247    4.247 aco_tsp_oo.py:112(solve_tsp)
      500    0.153    0.000    4.022    0.008 aco_tsp_oo.py:81(_colony_run)
     2500    0.324    0.000    3.865    0.002 aco_tsp_oo.py:151(walk_over_graph)
     2500    0.598    0.000    3.141    0.001 utils.py:26(create_dic_dist_from_graph)
   767500    1.486    0.000    1.739    0.000 defmatrix.py:189(__getitem__)
     2500    0.007    0.000    0.799    0.000 convert_matrix.py:442(to_numpy_matrix)
     2500    0.621    0.000    0.726    0.000 convert_matrix.py:1093(to_numpy_array)
    400

Y observamos que lo que mas se tarda son (la segunda columna es la que indica el tiempo total de ejecución) : 


     2500    0.324    0.000    3.865    0.002 aco_tsp_oo.py:151(walk_over_graph)
     2500    0.598    0.000    3.141    0.001 utils.py:26(create_dic_dist_from_graph)
   767500    1.486    0.000    1.739    0.000 create_dic_dist_from_graph(self.graph)
     2500    0.621    0.000    0.726    0.000 convert_matrix.py:1093(to_numpy_array)


In [28]:
cprof.dump_stats("solve_tsp_stats")


In [29]:
p_solve_tsp_stats = pstats.Stats("solve_tsp_stats")
print(p_solve_tsp_stats.sort_stats("cumulative").print_stats(10))

Wed May  5 12:18:30 2021    solve_tsp_stats

         3141046 function calls (3140546 primitive calls) in 4.248 seconds

   Ordered by: cumulative time
   List reduced from 69 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    4.247    2.124 /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3288(run_code)
        2    0.000    0.000    4.247    2.124 {built-in method builtins.exec}
        1    0.000    0.000    4.247    4.247 <ipython-input-27-5fb9737eed66>:3(<module>)
        1    0.010    0.010    4.247    4.247 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:112(solve_tsp)
      500    0.153    0.000    4.022    0.008 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:81(_colony_run)
     2500    0.324    0.000    3.865    0.002 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:151(walk_over_graph)
     2500    0.598    0.000    3.141  

Nuevamente observamos que:

-walk over graph (método de ant colony)

-get item (el cual es un método de numpy) 

-create dict dist from graph (método de ant colony)

-convert_matrix (el cual es un método de networkx)

son las que mas se tardan 



- Nos concentraremos en perfilar los métodos de ant colony que más se están tardando ya que nuestro objetivo es mejorar la implementacion del algoritmo 

# B. Perfilamiento en uso de memoria

In [68]:
#t = (colonia.solve_tsp())
#print(memory_usage(t, max_usage=True))

In [ ]:
#start_mem = memory_usage(max_usage=True)
#res = memory_usage(t, max_usage=True, retval=True)
#print('start mem', start_mem)
#print('max mem', res[0])
#print('used mem', res[0]-start_mem)
#print('fun output', res[1])

por algun motivo, el codigo vinculado a memory_usage no funciona ni en la computadora local ni en la remota, para nuestros métodos 

In [33]:
%load_ext memory_profiler


In [34]:
%memit #how much RAM this process is consuming


peak memory: 164.52 MiB, increment: 0.01 MiB


In [35]:
%memit -c colonia.solve_tsp()


peak memory: 292.00 MiB, increment: 127.47 MiB


## Perfilamiento Linea Por Linea 

A continuación, utilizamos la siguiente estrategia. Comenzamos con un perfilamiento de solve_tsp, por ser el método más general, y luego iremos perfilando submétodos y funciones que encontremos que están ocupando mucho tiempo y requieren un perfilamiento linea por linea

### Perfilamos el método solve_tsp

In [42]:
line_prof = line_profiler.LineProfiler()
print(line_prof(colonia.solve_tsp)())

None


In [43]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 8.97412 s
File: /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py
Function: solve_tsp at line 112

Line #      Hits         Time  Per Hit   % Time  Line Contents
   112                                               def solve_tsp(self):
   113                                                   """Resuelve el problema TSP.
   114                                                   """
   115         1          2.0      2.0      0.0          route = self.best_route
   116         1          1.0      1.0      0.0          dist = self.best_dist
   117                                                   
   118       501        940.0      1.9      0.0          for k in range(self.max_iter):
   119       500        478.0      1.0      0.0              A = atraccion_nodos(self.graph,tau= self.tau, eta=self.eta, 
   120       500     255796.0    511.6      2.9                                  alpha=self.alpha, beta=self.beta)
   121                     

Dado que observamos que self._colony_run(A) se tarda bastatante, vamos a analizar ese método por separado
Tambien vamos a perfilar la funcion atraccion_nodos 

### Perfilamos la funcion atraccion_nodos

In [48]:
tau = init_ferom(G)
eta = init_atrac(G, create_dic_dist_from_graph(G))

A = atraccion_nodos(G, tau, eta, alpha=1, beta=5) 

In [56]:
line_prof = line_profiler.LineProfiler()
print(line_prof(atraccion_nodos)(G, tau, eta, alpha=1, beta=5))

In [55]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.000679 s
File: /usr/local/lib/python3.6/dist-packages/ant_colony/utils.py
Function: atraccion_nodos at line 91

Line #      Hits         Time  Per Hit   % Time  Line Contents
    91                                           def atraccion_nodos(G, tau, eta, alpha=1, beta=5):
    92                                               """Calcula el grado de atracción de todos los nodos pertenicientes al 
    93                                               grafo G.
    94                                           
    95                                               Args:
    96                                                   G (networkx graph): Grafo con relaciones asociadas entre nodos
    97                                                   tau (dic): Diccionario con niveles de feromonas de los vecinos de 
    98                                                   cada nodo
    99                                                   eta (dic): Diccionario con

- Observamos estos fors anidados que están demandando una mayor ejecución en tiempo en particular en la linea 117

### Perfilamos el metodo self._colony_run()

In [51]:
line_prof = line_profiler.LineProfiler()
print(line_prof(colonia._colony_run)(A))

None


In [52]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.007305 s
File: /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py
Function: _colony_run at line 81

Line #      Hits         Time  Per Hit   % Time  Line Contents
    81                                               def _colony_run(self, A):
    82                                                   """La hormigas de la colonia realizan recorridos 
    83                                                   independientes simultáneamente.
    84                                           
    85                                                   Args:
    86                                                       A (dic): nivel de atracción de los nodos con respecto
    87                                                       a sus vecinos.
    88                                                   """
    89         1          2.0      2.0      0.0          distances = []
    90         1          1.0      1.0      0.0          routes = []
    91

- Observamos que la mayor parte del tiempo de ejecucion se la lleva el metodo walk_over_graph, entonces procedemos  a hacer un perfilamiento del mismo 

### Perfilamos walk_over_graph

In [57]:
# 5 es la cantidad de hormigas
ants = [ant(G) for i in range(5)]

In [60]:
# tomamos una hormiga 
ant= ants[1]

In [66]:
lenghts= create_dic_dist_from_graph(G)
#7 era el nodo inicial que habiamos elegido mas arriba
init_node = 7

In [67]:
ant.walk_over_graph(init_node=init_node, 
                                dist = lenghts, 
                                atrac = A)

In [71]:
line_prof = line_profiler.LineProfiler()
print(line_prof(ant.walk_over_graph)(init_node=init_node, 
                                dist = lenghts, 
                                atrac = A))

None


In [72]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.001602 s
File: /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py
Function: walk_over_graph at line 151

Line #      Hits         Time  Per Hit   % Time  Line Contents
   151                                               def walk_over_graph(self, 
   152                                                                 init_node,
   153                                                                 dist, 
   154                                                                 atrac):
   155                                                   """La hormiga intenta recorrer el grafo y volver
   156                                                   al origen sin repetir otros nodos.
   157                                           
   158                                                   Args:
   159                                                       init_node (int): Nodo inicial del recorrido.
   160                                          

- Obvservamos que create_dic_dist_from_graph es lo que más se tarda y lo perfilamos a continuación 

### Perfilamos create_dic_dist_from_graph(G)

In [76]:
line_prof = line_profiler.LineProfiler()
print(line_prof(create_dic_dist_from_graph)(G))

In [75]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.001454 s
File: /usr/local/lib/python3.6/dist-packages/ant_colony/utils.py
Function: create_dic_dist_from_graph at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def create_dic_dist_from_graph(G):
    27                                               """Crea diccionario de distancias entre nodos a partir de un grafo. 
    28                                           
    29                                               Args:
    30                                                   G (networkx graph): Grafo con relaciones asociadas entre nodos
    31                                           
    32                                               Returns:
    33                                                   (dic): Diccionario de distancias de los nodos
    34                                               """
    35         1         13.0     13.0      0.9      nodos = list(G.node

Observamos que el mayor tiempo se está tardando en : 

- nx.to_numpy_matriz, un método del paquete networks

- el doble for en las lineas 38 a 41


In [ ]:
## Conclusion General 

## Conclusion General

Nuestro análisis comenzó por un perfilamiento general, y luego fuimos analizando los submétodos y funciones que más estaban requiriendo un tiempo de ejecución 

De este análisis se desprende que el mayor tiempo de ejecución se encuentra en:

- el for anidado en la función atraccion_nodos

- el for anidado en la función create_dic_dist_from_graph

Esta última función es particularmente importante dado que es llamada de manera muy reiterada en el algoritmo, cada vez que una hormiga debe decidir a dónde moverse. 
    
    
    